<a href="https://colab.research.google.com/github/jessalynlc/IDS705_final_project/blob/main/data_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import time
import requests
import json
import pandas as pd
import numpy as np

In [32]:
api_key = 'd70a0d83e1a942578e16b2ed70dab42f'

#pulling data for ERCOT
ercot_load_forecast_dam = 'https://api.gridstatus.io/v1/datasets/ercot_load_forecast_dam/query?api_key=' + api_key + '&limit=1000'
load = 'https://api.gridstatus.io/v1/datasets/ercot_load/query?api_key=' + api_key + '&limit=1000'
ercot_unplanned_resource_outages = 'https://api.gridstatus.io/v1/datasets/ercot_unplanned_resource_outages/query?api_key=' + api_key + '&limit=1000'
ercot_fuel_mix = 'https://api.gridstatus.io/v1/datasets/ercot_fuel_mix/query?api_key=' + api_key + '&limit=1000'
ercot_solar_actual_and_forecast_hourly = 'https://api.gridstatus.io/v1/datasets/ercot_solar_actual_and_forecast_hourly/query?api_key=' + api_key + '&limit=1000'
ercot_temperature_forecast_by_weather_zone = 'https://api.gridstatus.io/v1/datasets/ercot_temperature_forecast_by_weather_zone/query?api_key=' + api_key + '&limit=1000'
ercot_energy_storage_resources = 'https://api.gridstatus.io/v1/datasets/ercot_energy_storage_resources/query?api_key=' + api_key + '&limit=1000'


#LMP price pull (target)
#Pulling LMP at the average of all the hubs
ercot_lmp_by_settlement_point = 'https://api.gridstatus.io/v1/datasets/ercot_lmp_by_settlement_point/query/location/HB_HUBAVG?api_key=' + api_key + '&limit=1000'

#Other hubs incase we want to forecast them, but sticking with the bus average:HB_CENTRAL, HB_EAST, HB_HOUSTON, HB_BUSAVG, HB_NORTH, HB_PAN, HB_SOUTH HB_WEST

#Data Pull

In [34]:
WAIT_TIME = 1  # seconds

response = requests.get(ercot_load_forecast_dam)
ercot_load_forecast_dam_data = response.json()
time.sleep(WAIT_TIME)

response = requests.get(load)
load_data = response.json()
time.sleep(WAIT_TIME)

response = requests.get(ercot_unplanned_resource_outages)
ercot_unplanned_resource_outages_data = response.json()
time.sleep(WAIT_TIME)

response = requests.get(ercot_fuel_mix)
ercot_fuel_mix_data = response.json()
time.sleep(WAIT_TIME)

response = requests.get(ercot_solar_actual_and_forecast_hourly)
ercot_solar_actual_and_forecast_hourly_data = response.json()
time.sleep(WAIT_TIME)

response = requests.get(ercot_temperature_forecast_by_weather_zone)
ercot_temperature_forecast_by_weather_zone_data = response.json()
time.sleep(WAIT_TIME)

response = requests.get(ercot_energy_storage_resources)
ercot_energy_storage_resources_data = response.json()
time.sleep(WAIT_TIME)

response = requests.get(ercot_lmp_by_settlement_point)
ercot_lmp_by_settlement_point_data = response.json()

#Question: Not all the datasets start at the same time, but I almost wonder if it's interesting to see how the model performs as more data gets added as time progresses...

#Organizing data frame

In [50]:
#extracting the value of the data key to create a list of dictionaries
ercot_load_forecast_dam_records = ercot_load_forecast_dam_data['data']
load_records = load_data['data']
ercot_unplanned_resource_outages_records = ercot_unplanned_resource_outages_data['data']
ercot_fuel_mix_records = ercot_fuel_mix_data['data']
ercot_solar_actual_and_forecast_hourly_records = ercot_solar_actual_and_forecast_hourly_data['data']
ercot_temperature_forecast_by_weather_zone_records = ercot_temperature_forecast_by_weather_zone_data['data']
ercot_energy_storage_resources_records = ercot_energy_storage_resources_data['data']
ercot_lmp_by_settlement_point_records = ercot_lmp_by_settlement_point_data['data']

# Create separate data frames
ercot_load_forecast_dam_df = pd.DataFrame(ercot_load_forecast_dam_records)
load_df = pd.DataFrame(load_records)
ercot_unplanned_resource_outages_df = pd.DataFrame(ercot_unplanned_resource_outages_records)
ercot_fuel_mix_df = pd.DataFrame(ercot_fuel_mix_records)
ercot_solar_actual_and_forecast_hourly_df = pd.DataFrame(ercot_solar_actual_and_forecast_hourly_records)
ercot_temperature_forecast_by_weather_zone_df = pd.DataFrame(ercot_temperature_forecast_by_weather_zone_records)
ercot_energy_storage_resources_df = pd.DataFrame(ercot_energy_storage_resources_records)
ercot_lmp_by_settlement_point_records_df = pd.DataFrame(ercot_lmp_by_settlement_point_records)

# Put all DataFrames in a list
dataframes = [
    ercot_load_forecast_dam_df,
    load_df,
    #ercot_unplanned_resource_outages_df, commenting out for now, have to think about how to add this to master
    ercot_fuel_mix_df,
    ercot_solar_actual_and_forecast_hourly_df,
    ercot_temperature_forecast_by_weather_zone_df,
    ercot_energy_storage_resources_df,
    ercot_lmp_by_settlement_point_records_df
]

#Standardize timestamps and collect time deltas
min_intervals = []
for i in range(len(dataframes)):
  df = dataframes[i].copy()

  # Rename time_utc to interval_start_utc if present, I think this is only for ercot_temperature_forecast_by_weather_zone_df
  if 'time_utc' in df.columns:
    df = df.rename(columns={'time_utc': 'interval_start_utc'})

  # Convert timestamps
  df['interval_start_utc'] = pd.to_datetime(df['interval_start_utc'])

  # Convert and inspect time delta if end time exists
  if 'interval_end_utc' in df.columns:
    df['interval_end_utc'] = pd.to_datetime(df['interval_end_utc'])
    deltas = (df['interval_end_utc'] - df['interval_start_utc']).dropna()
    if not deltas.empty:
      min_intervals.append(deltas.min())

  dataframes[i] = df

#Find smallest interval (e.g., 5 minutes)
smallest_interval = min(min_intervals)
print(f"Smallest interval detected: {smallest_interval}")

#Create master time index
all_times = pd.concat([df['interval_start_utc'] for df in dataframes])
time_index = pd.date_range(start=all_times.min(), end=all_times.max(), freq=smallest_interval)
master_df = pd.DataFrame({'interval_start_utc': time_index})

#Drop original interval_end_utc and merge
for df in dataframes:
    if 'interval_end_utc' in df.columns:
        df = df.drop(columns='interval_end_utc')
    master_df = master_df.merge(df, on='interval_start_utc', how='left')

#Add consistent interval_end_utc
master_df['interval_end_utc'] = master_df['interval_start_utc'] + smallest_interval

print(master_df.head())
print(master_df.shape)

Smallest interval detected: 0 days 00:05:00
         interval_start_utc publish_time_utc_x  north_x  south  west_x  \
0 2010-11-30 06:00:00+00:00                NaN      NaN    NaN     NaN   
1 2010-11-30 06:00:00+00:00                NaN      NaN    NaN     NaN   
2 2010-11-30 06:00:00+00:00                NaN      NaN    NaN     NaN   
3 2010-11-30 06:00:00+00:00                NaN      NaN    NaN     NaN   
4 2010-11-30 06:05:00+00:00                NaN      NaN    NaN     NaN   

   houston  system_total  load  coal_and_lignite  hydro  ...  west_y  \
0      NaN           NaN   NaN               NaN    NaN  ...     NaN   
1      NaN           NaN   NaN               NaN    NaN  ...     NaN   
2      NaN           NaN   NaN               NaN    NaN  ...     NaN   
3      NaN           NaN   NaN               NaN    NaN  ...     NaN   
4      NaN           NaN   NaN               NaN    NaN  ...     NaN   

   total_charging  total_discharging  net_output         sced_timestamp_utc  \